# K2496 example

Quantify K2496, a Ba, Ti, Si and O containing engineered glass.

In [ ]:
# Load DrWatson project manager and enable custom environment
using DrWatson
@quickactivate("HyperspectraWithNeXL")
# Load 3rd party libraries for plotting and tabulation
using Gadfly, DataFrames
# Load the necessary NeXL libraries
using Revise
using NeXLSpectrum


path = joinpath(datadir(),"exp_raw","K2496")
# Read the spectra from disk
k2496 = loadspectrum.(joinpath(path,"K2496_$(i).msa") for i in 1:3)
# Plot them using a method of Gadfly.plot(...) specialized for Spectrum items
set_default_plot_size(8inch,3inch)
plot(k2496..., klms=[ n"O", n"Si", n"Ti", n"Ba" ], xmax=6.0e3)

Define a BasicEDS detector object with the properties of the measurement device.  Then apply the detector to the measured spectra.

In [ ]:
det = BasicEDS(4096, -480.40409, 5.00525, 132.0, 110, 
         Dict(KShell=>n"B", LShell=>n"Ca", MShell=>n"Cs"))
k2496 = map(s->apply(s,det), k2496)
det

Compute the filtered reference spectra from the standards.  In this case, we are using "BaCl2 std.msa" for all Ba lines and then we will use Sanbornite as a standard for O, Si and Ba.

In [ ]:
refs = references([
  reference(n"Si", joinpath(path,"Si std.msa"), mat"Si"),
  reference(n"Ba", joinpath(path,"BaCl2 std.msa"), mat"BaCl2"),
  reference(n"O", joinpath(path,"MgO std.msa"), mat"MgO"),
  reference(n"Ti", joinpath(path,"Ti std.msa"), mat"Ti") ], det)
ENV["columns"]=400
asa(DataFrame, refs)

Fit (fit_spectra(...)) and then matrix correct (quantify(...)) the unknown spectra.

If we fit these references to the K2496 directly and then quantify, the results are ??????

In [ ]:
fs=fit_spectrum(k2496, refs)
asa(DataFrame, fs, withUnc=true)

In [ ]:
q=quantify.(fs)
asa(DataFrame, q, nominal=parse(Material,"0.323*O+0.2291*Si+0.018*Ti+0.4299*Ba",name="K2496 nominal"))

Now we turn our attention to the standard "Sanbornite."  We will fit our references to Sanbornite.

In [ ]:
refs2 = references( [
  reference(n"Si", joinpath(path,"Si std.msa"), mat"Si"),
  reference(n"Ba", joinpath(path,"BaCl2 std.msa"), mat"BaCl2"),
  reference(n"O", joinpath(path,"MgO std.msa"), mat"MgO") ], det);
sanbornite = loadspectrum(joinpath(path,"Sanbornite std.msa"))
sanbornite_std = fit_spectrum(apply(sanbornite, det), refs2)
asa(DataFrame, [ sanbornite_std ], withUnc=true)

In [ ]:
fs_stds = [ standardize(fsi, sanbornite_std, mat"BaSi2O5") for fsi in fs]
asa(DataFrame, fs_stds, withUnc=true)

In [ ]:
krs=kratios(sanbornite_std)

In [ ]:
krs[1].unkProps

In [ ]:
q_stds = quantify.(fs_stds)
asa(DataFrame, q_stds, nominal=parse(Material,"0.323*O+0.2291*Si+0.018*Ti+0.4299*Ba",name="K2496 nominal"))

fs is a vector of FilterFitResult items.  Plotting a FilterFitResult shows the raw spectral data (blue), the residual spectrum (red) and the rois and k-ratios in grey above the peaks.

In [ ]:
display(plot(fs[1]))

In [ ]:
display(plot(fs_stds[1]))

In [ ]:
asa(DataFrame, fs_stds[1])

The plot may be saved in various formats (including SVG, PNG and PDF).

In [ ]:
using Cairo, Fontconfig
plot(fs_stds[1]) |> SVG(joinpath(plotsdir(), "K2496 residual - sanbornite using BaCl2.svg"), 8inch, 3inch)
plot(fs_stds[1]) |> PNG(joinpath(plotsdir(), "K2496 residual - sanbornite using BaCl2.png"), 8inch, 3inch)
plot(fs_stds[1]) |> PDF(joinpath(plotsdir(), "K2496 residual - sanbornite using BaCl2.pdf"), 8inch, 3inch)

Output the quant results.  Write the results to a CSV file.

In [ ]:
df = asa(DataFrame, q, nominal=parse(Material,"0.323*O+0.2291*Si+0.018*Ti+0.4299*Ba",name="K2496 nominal"))
using CSV
CSV.write(joinpath(datadir(),"exp_pro","K2496 quant - sanbornite using BaCl2.csv"), df)
df

In [ ]:
describe(asa(DataFrame, q)[:,2:end], :mean, :std)

Tabulating a single IterationResult item provides extensive analysis details including uncertainties and matrix correction factors.

In [ ]:
asa(DataFrame, q[1])

In [ ]:
using PrettyTables
tbl = vcat( (asa(DataFrame,qi) for qi in q)...) 
open(joinpath(papersdir(),"Tables","K2496 results - sanbornite using BaCl2.ltx"),"w") do io
  pretty_table(io, tbl, nosubheader=true, backend=:latex, 
            label="tbl:k2496sanbornite")
end
tbl

QED